PatternMind — Visual Space Organization
This notebook contains all code for the PatternMind project. Text and code cells strictly alternate as required.

How to use:

Put your dataset on disk as folder-per-class.
Set DATASET_DIR below.
Run cells top-to-bottom.

In [ ]:
# === CONFIG ===
from pathlib import Path

# TODO: change this to your dataset root folder (one subfolder per class)
DATASET_DIR = Path("patternmind_dataset")

# Output folder for figures required by README
OUT_DIR = Path("images")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Reproducibility
SEED = 42


1) Imports and reproducibility
We import libraries and set seeds for reproducible feature extraction, sampling, and clustering

In [ ]:
import os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
from tqdm.auto import tqdm

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.preprocessing import StandardScaler

try:
    import umap
    UMAP_AVAILABLE = True
except Exception:
    UMAP_AVAILABLE = False

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


2) Dataset indexing
We scan the dataset directory and build a dataframe with image paths and labels. This lets us run EDA, sampling, and later analysis.

In [ ]:
from glob import glob

def index_image_folder(root: Path, exts=(".jpg", ".jpeg", ".png", ".bmp", ".webp")) -> pd.DataFrame:
    rows = []
    root = Path(root)
    assert root.exists(), f"DATASET_DIR not found: {root}"
    for class_dir in sorted([p for p in root.iterdir() if p.is_dir()]):
        label = class_dir.name
        for ext in exts:
            for fp in class_dir.rglob(f"*{ext}"):
                rows.append({"path": str(fp), "label": label})
    df = pd.DataFrame(rows)
    if df.empty:
        raise ValueError("No images found. Check DATASET_DIR and file extensions.")
    return df

df = index_image_folder(DATASET_DIR)
df.head(), df["label"].nunique(), len(df)


3) EDA (Exploratory Data Analysis)
We inspect class counts and visualize a few example images per category. This checks dataset balance and potential noise.

In [ ]:
# Class distribution
class_counts = df["label"].value_counts().sort_values(ascending=False)
display(class_counts.head(20))

#plt.figure()
#class_counts.plot(kind="bar")
#plt.title("Images per class")
#plt.ylabel("count")
#plt.tight_layout()
#plt.show()

counts = df["label"].value_counts()


print(f"Number of classes: {counts.shape[0]}")
print(f"Min images per class: {counts.min()}")
print(f"Max images per class: {counts.max()}")
print(f"Mean images per class: {counts.mean():.2f}")



Example grid per class (small sample)
We draw a simple preview grid for a subset of classes to ensure loading works and to understand intra-class variability.

In [ ]:
import math

def show_examples(df: pd.DataFrame, n_classes=6, n_per_class=6, figsize=(12, 8)):
    labels = list(df["label"].unique())[:n_classes]
    sample = (df[df["label"].isin(labels)]
              .groupby("label", group_keys=False)
              .apply(lambda x: x.sample(min(n_per_class, len(x)), random_state=SEED)))
    rows = len(labels)
    cols = n_per_class
    plt.figure(figsize=figsize)
    for i, (lbl, group) in enumerate(sample.groupby("label")):
        for j, (_, r) in enumerate(group.reset_index(drop=True).iterrows()):
            ax = plt.subplot(rows, cols, i*cols + j + 1)
            img = Image.open(r["path"]).convert("RGB")
            ax.imshow(img)
            ax.axis("off")
            if j == 0:
                ax.set_title(lbl, fontsize=10)
    plt.tight_layout()
    plt.show()

show_examples(df, n_classes=min(6, df["label"].nunique()), n_per_class=5)


4) Feature extraction (Deep CNN)
We use a pretrained CNN as a fixed feature extractor. For each image we obtain a vector representation from the penultimate layer. This representation is the basis for dimensionality reduction and clustering.

In [ ]:
# Image preprocessing for pretrained models
IMG_SIZE = 224
tfm = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, df: pd.DataFrame, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "path"]
        label = self.df.loc[idx, "label"]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label, path

# Pretrained backbone
backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# Remove classification head => output features (2048)
backbone.fc = nn.Identity()
backbone = backbone.to(device).eval()

@torch.no_grad()
def extract_cnn_features(df: pd.DataFrame, batch_size=64, num_workers=2):
    ds = ImageDataset(df, transform=tfm)
    dl = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=False,
                                     num_workers=num_workers, pin_memory=True)
    feats, labels, paths = [], [], []
    for xb, yb, pb in tqdm(dl, desc="Extracting CNN features"):
        xb = xb.to(device)
        fb = backbone(xb).detach().cpu().numpy()
        feats.append(fb)
        labels.extend(list(yb))
        paths.extend(list(pb))
    X = np.vstack(feats)
    return X, np.array(labels), np.array(paths)

# Optionally subsample for speed if dataset is huge
MAX_IMAGES = 1500
if len(df) > MAX_IMAGES:
    df_run = df.sample(MAX_IMAGES, random_state=SEED).reset_index(drop=True)
else:
    df_run = df.copy()

X_cnn, y, paths = extract_cnn_features(df_run, batch_size=64, num_workers=2)
X = X_cnn
X_cnn.shape


5) Baseline features (Color histograms)
As a classical baseline, we compute RGB color histograms. This often clusters by color themes (e.g., flags, sky, grass) and provides a useful contrast to CNN features

In [ ]:
def color_histogram(img: Image.Image, bins=16):
    arr = np.asarray(img.convert("RGB"))
    feat = []
    for c in range(3):
        h, _ = np.histogram(arr[:,:,c], bins=bins, range=(0,255), density=True)
        feat.append(h)
    return np.concatenate(feat).astype(np.float32)

def extract_color_hist_features(df: pd.DataFrame, bins=16):
    feats = []
    for p in tqdm(df["path"].tolist(), desc="Extracting color hist features"):
        img = Image.open(p).convert("RGB").resize((IMG_SIZE, IMG_SIZE))
        feats.append(color_histogram(img, bins=bins))
    return np.vstack(feats)

X_col = extract_color_hist_features(df_run, bins=16)
X_col.shape


## 6) Standardization
We standardize features before PCA/UMAP and clustering. This improves stability for distance-based methods.

In [ ]:
scaler_cnn = StandardScaler()
Xc = scaler_cnn.fit_transform(X_cnn)

scaler_col = StandardScaler()
Xh = scaler_col.fit_transform(X_col)

## 7) Dimensionality reduction (PCA / t-SNE / UMAP)
We compute low-dimensional embeddings for visualization. We will generate and save figures for the README in the `images/` folder.

In [ ]:
def embed_pca(X, n=2):
    return PCA(n_components=n, random_state=SEED).fit_transform(X)

def embed_tsne(X, n=2, perplexity=30):
    return TSNE(n_components=n, perplexity=perplexity, init="pca",
                learning_rate="auto", random_state=SEED).fit_transform(X)

def embed_umap(X, n=2, n_neighbors=15, min_dist=0.1):
    if not UMAP_AVAILABLE:
        raise ImportError("umap-learn not installed. Install it or skip UMAP.")
    return umap.UMAP(n_components=n, n_neighbors=n_neighbors, min_dist=min_dist,
                     random_state=SEED).fit_transform(X)

def plot_embedding(Z, labels, title, outpath=None, max_points=5000):
    # If many points, subsample for readability
    if len(Z) > max_points:
        idx = np.random.RandomState(SEED).choice(len(Z), size=max_points, replace=False)
        Zp = Z[idx]
        lp = labels[idx]
    else:
        Zp, lp = Z, labels

    plt.figure()
    # Map labels to integers for scatter (matplotlib handles categories poorly)
    lab_to_i = {lab:i for i, lab in enumerate(sorted(set(lp)))}
    c = np.array([lab_to_i[a] for a in lp])
    plt.scatter(Zp[:,0], Zp[:,1], c=c, s=8)
    plt.title(title)
    plt.xticks([]); plt.yticks([])
    plt.tight_layout()
    if outpath:
        plt.savefig(outpath, dpi=200)
    plt.show()

# CNN feature embeddings
Z_pca = embed_pca(Xc, 2)
plot_embedding(Z_pca, y, "PCA (CNN features)", OUT_DIR/"embedding_pca.png")

Z_tsne = embed_tsne(Xc, 2, perplexity=30)
plot_embedding(Z_tsne, y, "t-SNE (CNN features)", OUT_DIR/"embedding_tsne.png")

if UMAP_AVAILABLE:
    Z_umap = embed_umap(Xc, 2, n_neighbors=15, min_dist=0.1)
    plot_embedding(Z_umap, y, "UMAP (CNN features)", OUT_DIR/"embedding_umap.png")
else:
    Z_umap = None


## 8) Clustering
We apply several clustering algorithms to **organize** the visual space. We evaluate with internal metrics (Silhouette, Davies–Bouldin) and, for interpretation only, external agreement with labels (ARI/NMI).

In [ ]:
def eval_clustering(X, cluster_ids, labels_true=None):
    # Internal metrics (no labels needed)
    uniq = set(cluster_ids)
    sil = silhouette_score(X, cluster_ids) if len(uniq) > 1 else np.nan
    dbi = davies_bouldin_score(X, cluster_ids) if len(uniq) > 1 else np.nan

    out = {"n_clusters": len(uniq), "silhouette": sil, "davies_bouldin": dbi}

    # External metrics (only if labels are provided)
    if labels_true is not None:
        out["ARI"] = adjusted_rand_score(labels_true, cluster_ids)
        out["NMI"] = normalized_mutual_info_score(labels_true, cluster_ids)

    return out


results = []

# ------------------------------------------------------------
# More realistic K values: coarse visual groupings (not classes)
# ------------------------------------------------------------
K_LIST = [10, 15, 20, 30]

for k in K_LIST:
    # KMeans
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=SEED)
    cid_km = kmeans.fit_predict(Xc)
    results.append((f"KMeans(k={k})", eval_clustering(Xc, cid_km, y)))

    # Agglomerative
    agg = AgglomerativeClustering(n_clusters=k)
    cid_ag = agg.fit_predict(Xc)
    results.append((f"Agglomerative(k={k})", eval_clustering(Xc, cid_ag, y)))

# ------------------------------------------------------------
# DBSCAN: show that it often collapses without careful tuning
# ------------------------------------------------------------
for eps in [1.5, 2.0, 2.5, 3.0]:
    db = DBSCAN(eps=eps, min_samples=10)
    cid_db = db.fit_predict(Xc)
    results.append((f"DBSCAN(eps={eps})", eval_clustering(Xc, cid_db, y)))

pd.DataFrame([{**v, "method": m} for m, v in results]).set_index("method")



### Cluster visualization
We visualize cluster assignments on a 2D embedding (UMAP if available, else PCA). This is key for interpretability: compact clusters, overlaps, and outliers.

In [ ]:
# ============================================================
# CLUSTERING (fit in feature space X, visualize in 2D)
# ============================================================

def plot_clusters_on_embedding(
    Z, cluster_ids, title, outpath=None, max_points=5000, alpha=0.7
):
    if len(Z) > max_points:
        idx = np.random.RandomState(SEED).choice(len(Z), size=max_points, replace=False)
        Zp = Z[idx]
        cp = np.array(cluster_ids)[idx]
    else:
        Zp = Z
        cp = np.array(cluster_ids)

    plt.figure(figsize=(7, 6))
    plt.scatter(Zp[:, 0], Zp[:, 1], c=cp, s=8, alpha=alpha)
    plt.title(title)
    plt.xticks([]); plt.yticks([])
    plt.tight_layout()
    if outpath:
        plt.savefig(outpath, dpi=200)
    plt.show()


# ------------------------------------------------------------
# Choose visualization embedding (ONLY for plotting)
# ------------------------------------------------------------
Z_vis = Z_umap if Z_umap is not None else Z_pca


# ------------------------------------------------------------
# KMEANS (on 2048D features)
# ------------------------------------------------------------
k = 15  # intentionally small: visual archetypes, not classes
kmeans = KMeans(n_clusters=k, random_state=SEED, n_init=10)
cid_km = kmeans.fit_predict(X)


# ------------------------------------------------------------
# AGGLOMERATIVE (on 2048D features)
# ------------------------------------------------------------
agg = AgglomerativeClustering(n_clusters=k)
cid_ag = agg.fit_predict(X)


# ------------------------------------------------------------
# DBSCAN (on 2048D features)
# ------------------------------------------------------------
db = DBSCAN(eps=3.0, min_samples=10)
cid_db = db.fit_predict(X)


# ------------------------------------------------------------
# VISUALIZE CLUSTERS (computed in X, shown in Z)
# ------------------------------------------------------------
plot_clusters_on_embedding(
    Z_vis,
    cid_km,
    "KMeans clusters (fit in 2048D, shown in 2D)",
    OUT_DIR / "clusters_kmeans.png"
)

plot_clusters_on_embedding(
    Z_vis,
    cid_ag,
    "Agglomerative clusters (fit in 2048D, shown in 2D)",
    OUT_DIR / "clusters_agglomerative.png"
)

plot_clusters_on_embedding(
    Z_vis,
    cid_db,
    "DBSCAN clusters (fit in 2048D, shown in 2D)",
    OUT_DIR / "clusters_dbscan.png"
)



## 9) Cluster–Label relationship (ambiguity / purity)
Although labels are not used to train the clustering, they help **interpret** results. We compute a cluster×label table to find:
- clusters mixing multiple labels (ambiguity)
- labels split across many clusters (high intra-class variability)


In [ ]:
def cluster_label_table(cluster_ids, labels):
    tab = pd.crosstab(pd.Series(cluster_ids, name="cluster"),
                      pd.Series(labels, name="label"))
    return tab

tab_km = cluster_label_table(cid_km, y)
tab_km.head()


Heatmap-like visualization (simple)
We save a figure that communicates which labels dominate each cluster.

In [ ]:
def save_table_heatmap_like(tab: pd.DataFrame, title: str, outpath: Path, top_labels=20):
    # Optionally reduce columns for readability
    if tab.shape[1] > top_labels:
        cols = tab.sum(axis=0).sort_values(ascending=False).head(top_labels).index
        tab = tab[cols]
    plt.figure(figsize=(10, 6))
    plt.imshow(tab.values, aspect="auto")
    plt.title(title)
    plt.yticks(range(tab.shape[0]), tab.index)
    plt.xticks(range(tab.shape[1]), tab.columns, rotation=90)
    plt.tight_layout()
    plt.savefig(outpath, dpi=200)
    plt.show()

save_table_heatmap_like(tab_km, "Cluster × Label counts (KMeans)", OUT_DIR/"cluster_label_heatmap.png")


10) Nearest-neighbor analysis (semantic relationships)
To uncover relationships and ambiguities, we compute nearest neighbors in feature space. We then inspect cases where nearest neighbors belong to different labels, indicating visual similarity across categories.

In [ ]:
# ============================================================
# CLEAN SUMMARY (replaces the "Time series / Values / Faceted distributions" mess)
# ============================================================
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# --- Dataset summary (readable) ---
class_counts = df_run["label"].value_counts()
print(f"Images: {len(df_run)}")
print(f"Classes: {class_counts.shape[0]}")
print(f"Min / Mean / Max images per class: {class_counts.min()} / {class_counts.mean():.2f} / {class_counts.max()}")

display(class_counts.head(15).rename("count").to_frame())  # top-15 classes only

# Optional: simple histogram of class sizes (much more readable than 230 bars)
plt.figure(figsize=(7, 4))
plt.hist(class_counts.values, bins=30)
plt.title("Distribution of images per class")
plt.xlabel("Images per class")
plt.ylabel("Number of classes")
plt.tight_layout()
plt.show()

# --- Cluster purity summary (compact + meaningful) ---
def cluster_purity(cluster_ids, labels_true):
    tab = pd.crosstab(pd.Series(cluster_ids, name="cluster"), pd.Series(labels_true, name="label"))
    purity = (tab.max(axis=1) / tab.sum(axis=1)).sort_values(ascending=False)
    return purity

def show_purity(cluster_ids, name):
    purity = cluster_purity(cluster_ids, y)
    print(f"\n{name}:")
    print(f"  clusters: {purity.shape[0]}")
    print(f"  mean purity: {purity.mean():.3f}")
    print(f"  median purity: {purity.median():.3f}")
    display(purity.head(10).rename("purity").to_frame())  # top 10 purest clusters

# These variables should already exist from your clustering step:
# cid_km, cid_ag, cid_db
show_purity(cid_km, "KMeans purity")
show_purity(cid_ag, "Agglomerative purity")
# DBSCAN might produce -1 noise; purity still works, but often not very informative:
show_purity(cid_db, "DBSCAN purity")

# --- Nearest-neighbor cross-label rate (strong evidence) ---
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=6, metric="euclidean")
nn.fit(Xc)
dists, idxs = nn.kneighbors(Xc)

rows = []
for i in range(len(Xc)):
    for jpos in range(1, 6):  # skip self
        j = idxs[i, jpos]
        rows.append({
            "label_i": y[i],
            "label_j": y[j],
            "distance": float(dists[i, jpos]),
            "same_label": (y[i] == y[j]),
            "path_i": paths[i],
            "path_j": paths[j],
        })

nn_df = pd.DataFrame(rows)
cross_label_rate = (~nn_df["same_label"]).mean()
print(f"\nNearest-neighbor cross-label rate (k=5): {cross_label_rate:.3f}")

print("\nClosest cross-label neighbor pairs (top 10):")
display(nn_df[~nn_df["same_label"]].sort_values("distance").head(10))


Save a small nearest-neighbor gallery figure
We create a compact gallery for a few query images with their nearest neighbors. This directly supports the narrative about ambiguity and category overlap.

In [ ]:
def make_nn_gallery(sample_indices, idxs, paths, labels, outpath: Path, n_neighbors=6, thumb_size=128):
    cols = n_neighbors
    rows = len(sample_indices)
    fig_w = cols * 2
    fig_h = rows * 2
    plt.figure(figsize=(fig_w, fig_h))
    for r, i in enumerate(sample_indices):
        neigh = idxs[i, :n_neighbors]
        for c, j in enumerate(neigh):
            ax = plt.subplot(rows, cols, r*cols + c + 1)
            img = Image.open(paths[j]).convert("RGB").resize((thumb_size, thumb_size))
            ax.imshow(img)
            ax.axis("off")
            if c == 0:
                ax.set_title(f"Q: {labels[j]}", fontsize=9)
            else:
                ax.set_title(labels[j], fontsize=8)
    plt.tight_layout()
    plt.savefig(outpath, dpi=200)
    plt.show()

# pick a few diverse queries
rng = np.random.RandomState(SEED)
sample_indices = rng.choice(len(Xc), size=min(6, len(Xc)), replace=False)
make_nn_gallery(sample_indices, idxs, paths, y, OUT_DIR/"nn_gallery.png")
